#Considerações

1. Tensão de cisalhamento
==========================

##$\tau = \frac{FD}{2}$

##$\tau = \frac{8FD}{\pi d^3} + \frac{4F}{\pi d^2}$

##$\tau = K_s\frac{8FD}{\pi d^3}$

$K_s$ é um fator de correção da tensão de cisalhamento.
Usando o fator Bergsträsser, temos que:

##$\tau = K_B\frac{8FD}{\pi d^3}$
Onde,

$K_B = \frac{4C + 2}{4C - 3}$;

$C = \frac{D}{d}$, d = diâmetro do fio tabelado;

$4\leq C \leq 12$.


2. Ponto de operação
======================

O percurso fracionário até o fechamento da mola deve ser contido aos 75%.

##$F_s = (1 + \xi)F_{max}$
Onde
##$\xi  \geq 0.15$
Portanto, a tensão de cisalhamento:
##$\tau_s = K_B\frac{8(1 + \xi)F_{max}D}{\pi d^3}$

3. A cargo do projetista
========================

+ Escolher diâmetro $d$ do fio na tabela
+ O diâmetro médio “D” pode ser obtido a partir de $C$
+ Fator Bergsträsser $K_B$ é obtido a partir de $C$
+ $F_{max}$ ou $F$ são conhecidos
+ **Índice $C$ da mola**
+ **$\xi \geq 0.15$ deve ser seguido**

#Resistência a Compressão

Resistência a tração:

##$S_{ut} = \frac{A}{d^m}$

$S_{ut}$: Resistência à tração

𝐴 : Constante (obtida na Tabela)

𝑚: expoente (obtido na Tabela)

Resistência ao escoamento por cisalhamento:

##$S_{sy} = x * S_{ut}$

$x$: Constante da Tabela Porcentagem Máxima de resistência de Tração

1. Coeficiente de segurança
===========================

##$n_s = \frac{S_{sy}}{\tau_s}$

+ **$n_s$: Coeficiente de segurança deve ser $\geq 1.2$**

#Considerações Adicionais (opcional)

1. Rigidez da mola:

##$k = \frac{F}{y}$
e,

##$N_a = \frac{d^4G}{8D^3k}$

onde,

**$N_a$: Número de espiras ativas, deve respeitar $3 \leq N_a \leq 15$.**

$G$: Módulo de elasticidade transversal

In [2]:

import numpy as np

def getSpringDiam(springIndex, wireDiam):
  return springIndex * wireDiam*1000

def getK_Bergtrassen(C):
  return (4*C + 2) / (4*C - 3)

def getShearStress(BergtrassenF, Force, springDiam, wireDiam):
  return BergtrassenF * 8 * Force * springDiam / (np.pi * wireDiam**3)

def getTensileStrength(A, wireDiam, m):
  #'A' e 'm': valores tabelados
  #Encontrar o significado de 'A' e 'm'
  return (A / (wireDiam**m))

def getShearYieldStrenght(x, tensileStrenght):
  #'x': valor tabelado
  #Econtrar o significado de 'x'
  return (x * tensileStrenght)

def getSafetyFactor(ShearYS, tensileStrength):
  return ShearYS / tensileStrength

#Parametros utilizados com base no exemplo 10-2 do livro Shigley pág.538. No livro

d = 2.03*0.001
C = 10.5
F = 89
print(F)

D = getSpringDiam(C, d)
K_B = getK_Bergtrassen(C)
T_s = getShearStress(K_B, F, D*0.001, d)
print("Tensão de cisalhamento(MPa):", T_s/100000)
T_s = (1 + 0.15) * T_s
print("Diâmetro de mola(mm):", D)
print("Fator Bergstrassen:", K_B)
print("Tensão de cisalhamento corrigido (MPa):", T_s/1000000)

print()

A = 2211e6
m = 0.145
x = 0.45
S_ut = getTensileStrength(A, d*1000, m)
S_sy = getShearYieldStrenght(x, S_ut)
n_s = getSafetyFactor(S_sy, T_s)
print("Resistência à compressão(MPa):", S_ut/1000000)
print("Resistência a escoamento de cisahamento(MPa):", S_sy/1000000)
print("fator de segurança:", n_s)
if (n_s >= 1.2):
  print("Projeto não falha")
else: print("Projeto passível de falha")


89
Tensão de cisalhamento(MPa): 6515.014905310134
Diâmetro de mola(mm): 21.314999999999998
Fator Bergstrassen: 1.1282051282051282
Tensão de cisalhamento corrigido (MPa): 749.2267141106653

Resistência à compressão(MPa): 1995.2706631325634
Resistência a escoamento de cisahamento(MPa): 897.8717984096536
fator de segurança: 1.1983980035674922
Projeto passível de falha


In [1]:
import numpy as np
import tkinter as tk
from tkinter import ttk

############################################# Classe Material
class Material:
    def __init__(self, nome, diametros_fio, expoente_m, valor_A, resistencia_tracao_max):
        """
        Inicializa um novo objeto Material.
        :param id: identificação do material
        :param diametros_fio: Lista de valores de diâmetros de fio.
        :param expoente_m: Valor do expoente m.
        :param valor_A: Valor de A.
        :param resistencia_tracao_max: Porcentagem máxima de resistência de tração.
        """
        self.id = nome
        self.diametros_fio = diametros_fio
        self.expoente_m = expoente_m
        self.valor_A = valor_A
        self.resistencia_tracao_max = resistencia_tracao_max

    def __str__(self):
        return (f"Material {self.id} com diâmetros: {self.diametros_fio}, "
                f"m: {self.expoente_m}, A: {self.valor_A}, "
                f"Resistência de tração máxima: {self.resistencia_tracao_max}%")

# Exemplo de uso
material_exemplo = Material(
    nome = "A227",
    diametros_fio=[1.0, 1.5, 2.0],
    expoente_m=0.145,
    valor_A=2210,
    resistencia_tracao_max=45
)

print(material_exemplo)


############################################# Funções de cálculo mecânico
def getSpringDiam(springIndex, wireDiam):
  return springIndex * wireDiam*1000

def getK_Bergtrasser(C):
  return (4*C + 2) / (4*C - 3)

def getShearStress(BergtrassenF, Force, springDiam, wireDiam):
  return BergtrassenF * 8 * Force * springDiam / (np.pi * wireDiam**3)

def getTensileStrength(A, wireDiam, m):
  #'A' e 'm': valores tabelados
  #Encontrar o significado de 'A' e 'm'
  return (A / (wireDiam**m))

def getShearYieldStrenght(x, tensileStrenght):
  #'x': valor tabelado
  #Econtrar o significado de 'x'
  return (x * tensileStrenght)

def getSafetyFactor(ShearYS, tensileStrength):
  return ShearYS / tensileStrength

################################################ Funções da interface
def on_dropdown_materials_select(event):
  global selected_material
  selected_material = dropdown_material_var.get()

  new_diameters = materials.get(selected_material, [])
  dropdown_diameters_menu['values'] = new_diameters
  dropdown_diameter_var.set('')

def on_dropdown_diameter_select(event):
  selected_diameter = dropdown_diameter_var.get()

def on_slider_change(value):
  slider_value = slider_var.get()
  print("Slider value:", slider_value)

def on_number_entry():
  try:
    number_value = float(number_entry.get())
    print("Number value:", number_value)
  except ValueError:
    print("Invalid number input")

if __name__ == "__main__":
  root = tk.Tk()
  root.title('Projeto Slinky') 
  # root.geometry('800x600+100+0')


  materials = {
              "": [],
              "A227": [1, 2, 3, 4, 5],
              "A228": [1.1, 2.2, 3.3]
              }

  # Dropdown menu

  # Materials
  selected_material = ""
  # dropdown_materials = ["A227", "A228"]
  dropdown_materials = list(materials.keys())
  dropdown_material_var = tk.StringVar(value=selected_material)
  dropdown_materials_menu = ttk.Combobox(root, textvariable=dropdown_material_var, values=dropdown_materials)
  dropdown_materials_menu.bind("<<ComboboxSelected>>", on_dropdown_materials_select)
  dropdown_materials_menu.pack()

  # Diameters
  dropdown_diameter_var = tk.StringVar(value = "")
  dropdown_diameters = materials[selected_material]
  dropdown_diameters_menu = ttk.Combobox(root, textvariable=dropdown_diameter_var, values=dropdown_diameters)
  dropdown_diameters_menu.bind("<<ComboboxSelected>>", on_dropdown_diameter_select)
  dropdown_diameters_menu.pack()


  # Slider
  slider_var = tk.DoubleVar(value=4)
  slider = ttk.Scale(root, from_=4, to=12, orient="horizontal", variable=slider_var, command=on_slider_change)
  slider.pack()

  # Number entry
  number_entry = tk.Entry(root)
  number_entry.pack()
  number_button = tk.Button(root, text="Enter Number", command=on_number_entry)
  number_button.pack()

  root.mainloop()


Material A227 com diâmetros: [1.0, 1.5, 2.0], m: 0.145, A: 2210, Resistência de tração máxima: 45%
